## Régression logistique et MLflow

Nous allons exécuter notre code sur Databricks Community Edition, qui est l'option la plus facile pour appréhender l'apport de MLflow car elle comporte un service managé MLFlow.

Si vous lancez ce notebook en local, il faudra utiliser un serveur de tracking en local, etc. Le [guide suivant](https://www.mlflow.org/docs/latest/getting-started/running-notebooks/index.html) peut vous aider à faire le set-up. J'ai commenté les lignes de code qui vont être nécessaires en dehors de Databricks.

In [0]:
# Cellule d'import à compléter
import mlflow
# Votre code ici

In [0]:
import mlflow
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
# mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

Définir une expérimentation à l'aide de mlflow.set_experiment

In [0]:
# Votre code ici

In [0]:
mlflow.set_experiment("/Users/benjamin.speziale@soprasterianext.com/aaa")

2024/04/04 22:17:42 INFO mlflow.tracking.fluent: Experiment with name '/Users/benjamin.speziale@soprasterianext.com/aaa' does not exist. Creating a new experiment.


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/390552462623706', creation_time=1712269062566, experiment_id='390552462623706', last_update_time=1712269062566, lifecycle_stage='active', name='/Users/benjamin.speziale@soprasterianext.com/aaa', tags={'mlflow.experiment.sourceName': '/Users/benjamin.speziale@soprasterianext.com/aaa',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'benjamin.speziale@soprasterianext.com',
 'mlflow.ownerId': '1978442123578406'}>


Charger les données. Elles sont au format libsvm, qui est un format très utilisé pour représenter des données creuses (sparse). Elles sont déjà sous la forme attendue par un modèle (c'est-à-dire que dans ce TP, il n'y aura pas besoin de VectorAssembler).

In [0]:
# Votre code ici

In [0]:
# load data file.
inputData = spark.read.format("libsvm") \
    .load("/FileStore/tables/sample_multiclass_classification_data.txt")

In [0]:
display(inputData)

label,features
1.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 1, 2, 3), values -> List(-0.222222, 0.5, -0.762712, -0.833333))"
1.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 1, 2, 3), values -> List(-0.555556, 0.25, -0.864407, -0.916667))"
1.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 1, 2, 3), values -> List(-0.722222, -0.166667, -0.864407, -0.833333))"
1.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 1, 2, 3), values -> List(-0.722222, 0.166667, -0.694915, -0.916667))"
0.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 1, 2, 3), values -> List(0.166667, -0.416667, 0.457627, 0.5))"
1.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 2, 3), values -> List(-0.833333, -0.864407, -0.916667))"
2.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 1, 2, 3), values -> List(-1.32455E-7, -0.166667, 0.220339, 0.0833333))"
2.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 1, 2, 3), values -> List(-1.32455E-7, -0.333333, 0.0169491, -4.03573E-8))"
1.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 1, 2, 3), values -> List(-0.5, 0.75, -0.830508, -1.0))"
0.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0, 2, 3), values -> List(0.611111, 0.694915, 0.416667))"


En utilisant le TP précédent, implémenter une classification multiclasse en utilisant un classifier OneVsRest.

Si vous êtes à l'aise, à réaliser en même temps, sinon après : logger dans MLflow les hyperparamètres, la métrique, et ajouter un tag pour décrire le run.

In [0]:
# with mlflow.start_run():
#    # Votre code ici

In [0]:
# Votre code ici

In [0]:
(train, test) = inputData.randomSplit([0.8, 0.2])

params = {
    "maxIter": 10,
    "tol": 1E-5,
    "fitIntercept": False
}
mlflow.log_params(params)

lr = LogisticRegression(**params)

ovr = OneVsRest(classifier=lr)
ovrModel = ovr.fit(train)
predictions = ovrModel.transform(test)

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
mlflow.log_metric("accuracy", accuracy)
print("Test Error = %g" % (1.0 - accuracy))

mlflow.set_tag("Training Info", "Basic LR model for iris data")

Test Error = 0.241379
